In [1]:
import pandas as pd
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import print_breakdown, search

# TODO: handle non-US cars
df_nhtsa = await search(
  # min_model_year=2017,
  include_openpilot=True,
  include_police=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36766 VINs (filter_comment=None, include_openpilot=True, skipped=162, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/36766 [00:00<?, ?it/s]


Model
                      2
Aviator            1107
Bronco              997
Bronco Sport       1500
C-Max                 5
Continental          23
Corsair             986
E-Transit           157
Ecosport            406
Edge               2685
Escape             3569
Expedition          835
Expedition MAX      733
Explorer           3364
F-150              8254
F-150 Lightning     385
F-250              1720
F-350              1158
F-450               207
F-550                 1
Fiesta              196
Flex                189
Focus               169
Fusion             1163
GT                    3
MKC                  55
MKT                   7
MKZ                 100
Maverick            994
Mustang             952
Mustang Mach-E      839
Nautilus           1169
Navigator           303
Navigator L         230
Ranger              533
Taurus              122
Transit            1234
Transit Connect     414
dtype: int64


In [2]:
from panda.python.uds import DATA_IDENTIFIER_TYPE
from notebooks.ford.ecu import FordEcu


def get_platform_code(row):
  abd = AsBuiltData.from_vin(row['VIN'])

  fw = abd.get_identifier(FordEcu.AntiLockBrakeSystem, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  # fw = abd.get_identifier(FordEcu.PowerSteeringControlModule, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  if fw is None:
    platform_code = None
  else:
    prefix, core, _ = fw.split('-')
    platform_code = f'{core}-{prefix[1:]}'

  return pd.Series({
    'PlatformCode': platform_code,
    'PlatformYear': prefix[0] if platform_code else None,
  })


df = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear']].copy()
df['CarName'] = df['Make'] + ' ' + df['Model'] + ' ' + df['ModelYear'].astype(str)
df.drop(columns=['Make', 'Model', 'ModelYear'], inplace=True)
df = df.join(df_nhtsa.apply(get_platform_code, axis=1))
df.head()

,VIN,CarName,PlatformCode,PlatformYear
0,1FTEW1E87NFB26658,FORD F-150 2022,2D053-L3V,N
1,2FMPK4J90LBA41465,FORD Edge 2020,2D053-2GC,L
2,1FT6W1EV4PWG01115,FORD F-150 Lightning 2023,2D053-L38,P
3,2FMPK4K92NBA10882,FORD Edge 2022,2D053-2GC,M
4,1FTEW1EP3PKF28680,FORD F-150 2023,2D053-L34,P


In [3]:
# which cars have the same platform code
for group in df.groupby(['PlatformCode', 'PlatformYear'], dropna=False):
  print(group[0])
  print(f'CarName: {group[1]["CarName"].unique()}')
  print()

('14C036-1FC', 'F')
CarName: ['FORD Focus 2018' 'FORD Focus 2017']

('14C036-B3C', 'K')
CarName: ['FORD Ranger 2019' 'FORD Ranger 2023' 'FORD Ranger 2020'
 'FORD Ranger 2021' 'FORD Ranger 2022']

('14C036-K41', 'F')
CarName: ['FORD Transit 2019' 'FORD Transit 2018']

('14C036-V61', 'C')
CarName: ['FORD Escape 2015']

('14C036-V61', 'G')
CarName: ['FORD Escape 2018' 'FORD Escape 2019' 'FORD Transit Connect 2018']

('2D053-1MC', 'L')
CarName: ['FORD Explorer 2023' 'FORD Explorer 2021' 'FORD Explorer 2020'
 'LINCOLN Aviator 2020' 'FORD Explorer 2022' 'LINCOLN Aviator 2023'
 'LINCOLN Aviator 2021' 'FORD Explorer 2024' 'LINCOLN Aviator 2022'
 'LINCOLN Aviator 2024']

('2D053-2BC', 'D')
CarName: ['FORD Fiesta 2019' 'FORD Fiesta 2018' 'FORD Fiesta 2017']

('2D053-2GC', 'F')
CarName: ['FORD Edge 2018']

('2D053-2GC', 'K')
CarName: ['FORD Edge 2019' 'LINCOLN Nautilus 2019']

('2D053-2GC', 'L')
CarName: ['FORD Edge 2020' 'LINCOLN Nautilus 2020']

('2D053-2GC', 'M')
CarName: ['FORD Edge 2022' 'FO

In [4]:
from notebooks.ford.platforms import find_openpilot_platform


def get_openpilot_platform(row):
  return pd.Series({
    'CarInfoPlatform': find_openpilot_platform(row['CarName']),
  })


df_openpilot_platforms = df.join(df.apply(get_openpilot_platform, axis=1))
df_openpilot_platforms.head()

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


,VIN,CarName,PlatformCode,PlatformYear,CarInfoPlatform
0,1FTEW1E87NFB26658,FORD F-150 2022,2D053-L3V,N,None
1,2FMPK4J90LBA41465,FORD Edge 2020,2D053-2GC,L,None
2,1FT6W1EV4PWG01115,FORD F-150 Lightning 2023,2D053-L38,P,FORD F-150 LIGHTNING 1ST GEN
3,2FMPK4K92NBA10882,FORD Edge 2022,2D053-2GC,M,None
4,1FTEW1EP3PKF28680,FORD F-150 2023,2D053-L34,P,FORD F-150 14TH GEN


In [5]:
df_openpilot_platforms['CarInfoPlatform'].value_counts(dropna=False)

CarInfoPlatform
None                            26681
FORD EXPLORER 6TH GEN            3678
FORD ESCAPE 4TH GEN              2164
FORD F-150 14TH GEN              1338
FORD BRONCO SPORT 1ST GEN         930
FORD MUSTANG MACH-E 1ST GEN       813
FORD MAVERICK 1ST GEN             788
FORD F-150 LIGHTNING 1ST GEN      374
Name: count, dtype: int64

In [6]:
for group in df_openpilot_platforms.groupby('CarInfoPlatform', dropna=False):
  print(group[0])
  print(f'CarName: {group[1]["CarName"].unique()}')
  print(f'PlatformCode: {group[1]["PlatformCode"].unique()}')
  print(f'PlatformYear: {group[1]["PlatformYear"].unique()}')
  print()

FORD BRONCO SPORT 1ST GEN
CarName: ['FORD Bronco Sport 2021' 'FORD Bronco Sport 2022']
PlatformCode: ['2D053-X6C']
PlatformYear: ['L']

FORD ESCAPE 4TH GEN
CarName: ['FORD Escape 2021' 'FORD Escape 2020' 'FORD Escape 2022']
PlatformCode: ['2D053-X6C']
PlatformYear: ['L']

FORD EXPLORER 6TH GEN
CarName: ['FORD Explorer 2023' 'FORD Explorer 2021' 'FORD Explorer 2020'
 'LINCOLN Aviator 2020' 'FORD Explorer 2022' 'LINCOLN Aviator 2023'
 'LINCOLN Aviator 2021' 'LINCOLN Aviator 2022']
PlatformCode: ['2D053-1MC']
PlatformYear: ['L']

FORD F-150 14TH GEN
CarName: ['FORD F-150 2023']
PlatformCode: ['2D053-L34' '2D053-L3V']
PlatformYear: ['P']

FORD F-150 LIGHTNING 1ST GEN
CarName: ['FORD F-150 Lightning 2023' 'FORD F-150 Lightning 2022']
PlatformCode: ['2D053-L38']
PlatformYear: ['P' 'N']

FORD MAVERICK 1ST GEN
CarName: ['FORD Maverick 2022' 'FORD Maverick 2023']
PlatformCode: ['2D053-Z6C']
PlatformYear: ['N' 'P']

FORD MUSTANG MACH-E 1ST GEN
CarName: ['FORD Mustang Mach-E 2021' 'FORD Mustang M

In [7]:
# which openpilot platforms have the same platform code
for group in df_openpilot_platforms.groupby(['PlatformCode', 'PlatformYear'], dropna=False):
  platforms = group[1]['CarInfoPlatform'].unique()
  if len(platforms) == 1 and platforms[0] is None:
    continue
  print(group[0])
  print(f'CarInfoPlatform: {group[1]["CarInfoPlatform"].unique()}')
  print(f'CarName: {group[1]["CarName"].unique()}')
  print()

('2D053-1MC', 'L')
CarInfoPlatform: [<CAR.EXPLORER_MK6: 'FORD EXPLORER 6TH GEN'> None]
CarName: ['FORD Explorer 2023' 'FORD Explorer 2021' 'FORD Explorer 2020'
 'LINCOLN Aviator 2020' 'FORD Explorer 2022' 'LINCOLN Aviator 2023'
 'LINCOLN Aviator 2021' 'FORD Explorer 2024' 'LINCOLN Aviator 2022'
 'LINCOLN Aviator 2024']

('2D053-J9C', 'L')
CarInfoPlatform: [<CAR.MUSTANG_MACH_E_MK1: 'FORD MUSTANG MACH-E 1ST GEN'>]
CarName: ['FORD Mustang Mach-E 2021']

('2D053-K9C', 'L')
CarInfoPlatform: [<CAR.MUSTANG_MACH_E_MK1: 'FORD MUSTANG MACH-E 1ST GEN'> None]
CarName: ['FORD Mustang Mach-E 2021' 'FORD Mustang Mach-E 2022'
 'FORD Mustang Mach-E 2023' 'FORD Mustang Mach-E 2024']

('2D053-L34', 'P')
CarInfoPlatform: [<CAR.F_150_MK14: 'FORD F-150 14TH GEN'> None]
CarName: ['FORD F-150 2023' 'FORD F-150 2022' 'FORD F-150 2021']

('2D053-L38', 'N')
CarInfoPlatform: [<CAR.F_150_LIGHTNING_MK1: 'FORD F-150 LIGHTNING 1ST GEN'>]
CarName: ['FORD F-150 Lightning 2022']

('2D053-L38', 'P')
CarInfoPlatform: [<CA

In [8]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df_openpilot_platforms.groupby(['PlatformCode', 'PlatformYear'], dropna=False).agg(lambda x: ', '.join(sorted((str(x) for x in set(x)))))[['CarName', 'CarInfoPlatform']]

CarName  \
PlatformCode PlatformYear                                                                                                                                                                                                                                                         
14C036-1FC   F                                                                                                                                                                                                                                 FORD Focus 2017, FORD Focus 2018   
14C036-B3C   K                                                                                                                                                                         FORD Ranger 2019, FORD Ranger 2020, FORD Ranger 2021, FORD Ranger 2022, FORD Ranger 2023   
14C036-K41   F                                                                                                                                                                                                                             FORD Transit 2018, FORD Transit 2019   
14C036-V61   C                                                                                                                                                                                                                                                 FORD Escape 2015   
             G                                                                                                                                                                                                    FORD Escape 2018, FORD Escape 2019, FORD Transit Connect 2018   
2D053-1MC    L                                                 FORD Explorer 2020, FORD Explorer 2021, FORD Explorer 2022, FORD Explorer 2023, FORD Explorer 2024, LINCOLN Aviator 2020, LINCOLN Aviator 2021, LINCOLN Aviator 2022, LINCOLN Aviator 2023, LINCOLN Aviator 2024   
2D053-2BC    D                                                                                                                                                                                                             FORD Fiesta 2017, FORD Fiesta 2018, FORD Fiesta 2019   
2D053-2GC    F                                                                                                                                                                                                                                                   FORD Edge 2018   
             K                                                                                                                                                                                                                            FORD Edge 2019, LINCOLN Nautilus 2019   
             L                                                                                                                                                                                                                            FORD Edge 2020, LINCOLN Nautilus 2020   
             M                                                                                                                                                                                     FORD Edge 2021, FORD Edge 2022, LINCOLN Nautilus 2021, LINCOLN Nautilus 2022   
             N                                                                                                                                                                                                                            FORD Edge 2022, LINCOLN Nautilus 2022   
             P                                                                                                                                                                                                            FORD Edge 2023, FORD Edge 2024, LINCOLN Nautilus 2023   
2D053-3GC    G                                                                                                                             